# 라이브러리

In [2]:
import numpy as np
import torch
from __future__ import print_function
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch.nn.functional as F
import time

from google.colab import drive
drive.mount('/content/drive')

device = 'cuda' if cuda.is_available() else 'cpu'
path = "/content/drive/MyDrive/21_winter/CIFAR-10/cifar-10-batches-py"
batch_size = 32

Mounted at /content/drive


# 데이터 불러오기

In [3]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
data_list = ["/data_batch_" + str(i) for i in range(2,6)]
train_data = unpickle(path + "/data_batch_1")
test_data = unpickle(path + "/test_batch")

train_data_img = train_data[b'data']
train_data_labels = train_data[b'labels']
for i in data_list:
  train_data = unpickle(path + i)
  train_data_img = np.concatenate((train_data_img,train_data[b'data']))
  train_data_labels += train_data[b'labels']


# 데이터 준비

In [4]:
class GH_Dataset_train(data.Dataset): 
    def __init__(self,X,Y):
        self.x_data = torch.from_numpy(X).type(dtype=torch.float32).resize_((50000,3,32,32))
        self.y_data = torch.tensor(Y).resize_(50000,1)

    def __len__(self): 
        return len(self.x_data)
    
    def __getitem__(self, idx): 
        x = self.x_data[idx]
        y = self.y_data[idx]
        return x, y

class GH_Dataset_test(data.Dataset): 
    def __init__(self,X,Y):
        self.x_data = torch.from_numpy(X).type(dtype=torch.float32).resize_((10000,3,32,32))
        self.y_data = torch.tensor(Y).resize_(10000,1)

    def __len__(self): 
        return len(self.x_data)
    
    def __getitem__(self, idx): 
        x = self.x_data[idx]
        y = self.y_data[idx]
        return x, y

train_dataset = GH_Dataset_train(train_data_img,train_data_labels)
train_loader = data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

test_dataset = GH_Dataset_test(test_data[b'data'],test_data[b'labels'])
test_loader = data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

# 모델

In [5]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, torch.squeeze(target))
        loss.backward()
        optimizer.step()
        if batch_idx %100000 == 0:
            print('==================\nTrain Epoch : {} | Batch Status : {}/{} ({:.0f}%) | Loss : {:.6f}'.format(
                epoch, batch_idx*len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, torch.squeeze(target)).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    test_loss /= len(test_loader.dataset)
    print(f'Test set: Average loss : {test_loss:.4f}, Accuracy : {correct}/{len(test_loader.dataset)}'
          f'({100. * correct / len(test_loader.dataset):.0f}%)')

Downloading: "https://github.com/pytorch/vision/archive/v0.10.0.zip" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

# 모델 학습

In [6]:
if __name__ == '__main__':
    since = time.time()
    for epoch in range(1, 10):
        epoch_start = time.time()
        train(epoch)
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Training time: {m:.0f}m {s:.0f}s')
        
        test()
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Tesing time: {m:.0f}m {s:.0f}s')
        
    m, s = divmod(time.time() - epoch_start, 60)
    print(f'Total time : {m:.0f}m {s: .0f}s \nModel was trained on {device}!')

Train Epoch : 1 | Batch Status : 0/50000 (0%) | Loss : 12.043117
Training time: 1m 20s
Test set: Average loss : 0.0199, Accuracy : 7852/10000(79%)
Tesing time: 1m 25s
Train Epoch : 2 | Batch Status : 0/50000 (0%) | Loss : 0.267860
Training time: 1m 20s
Test set: Average loss : 0.0174, Accuracy : 8135/10000(81%)
Tesing time: 1m 25s
Train Epoch : 3 | Batch Status : 0/50000 (0%) | Loss : 0.302580
Training time: 1m 19s
Test set: Average loss : 0.0160, Accuracy : 8291/10000(83%)
Tesing time: 1m 24s
Train Epoch : 4 | Batch Status : 0/50000 (0%) | Loss : 0.463323
Training time: 1m 19s
Test set: Average loss : 0.0167, Accuracy : 8312/10000(83%)
Tesing time: 1m 24s
Train Epoch : 5 | Batch Status : 0/50000 (0%) | Loss : 0.071039
Training time: 1m 19s
Test set: Average loss : 0.0164, Accuracy : 8404/10000(84%)
Tesing time: 1m 24s
Train Epoch : 6 | Batch Status : 0/50000 (0%) | Loss : 0.067754
Training time: 1m 19s
Test set: Average loss : 0.0172, Accuracy : 8433/10000(84%)
Tesing time: 1m 24s
Tra